In [7]:
!pip install pmdarima

     |████████████████████████████████| 1.1MB 2.7MB/s 


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

data = pd.read_excel('United_Way_Synthetic_Dataset.xlsx')

def generate_annual_metric_visualization(strategy_name,program_name=None):
    strategy_frame = data[(data['Strategy_Name']==strategy_name)]
    if program_name is not None:
      strategy_frame = strategy_frame[strategy_frame['Program_Name']==program_name]

    metric_names = list(set(strategy_frame['Outcome_Indicator_Name']))
    metric_sums = []
    metric_targets =[]
    for metric_name in metric_names:
        metric_sum = 0
        metric_frame = strategy_frame[strategy_frame['Outcome_Indicator_Name']==metric_name]
        for i in range(-8,-1):
            metric_sum += metric_frame[metric_frame.columns[i]].sum()
        metric_sums.append(metric_sum)
        metric_targets.append(metric_frame[metric_frame.columns[-1]].sum())
    
    ind = np.arange(len(metric_names)) 
    width = 0.35       
    plt.bar(ind, metric_sums, width, label='Actual to Date')
    plt.bar(ind + width, metric_targets, width, label='Annual Target')
    
    plt.ylabel('Measure Value')
    plt.xticks(ind + width / 2, metric_names, rotation='vertical')
    plt.legend(loc='best')
    plt.show()

#generate_annual_metric_visualization('Expand Intervention Services for Family Violence', 'Agency A - Program I')

def generate_specific_metric_visualization(metric_name,strategy_name,program_name=None):
    frame = data[(data['Strategy_Name']==strategy_name)&(data['Outcome_Indicator_Name']==metric_name)]
    if program_name is not None:
      frame = frame[frame['Program_Name']==program_name]
    
    values = []
    cumulative_values = []
    x_axis = []
    target = frame[frame.columns[-1]].sum()
    
    for i in range(-8,-1):
        values.append(frame[frame.columns[i]].sum())
        x_axis.append(frame.columns[i])
    
    print(values)
    
    auto_model = auto_arima(values, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    
    #print(auto_model.order)
    #clear_output()
    nvalues = np.array(values, dtype=np.float32)
    model = ARIMA(nvalues, order=auto_model.order)
    fitted = model.fit(disp=-1)  
    
    # Forecast
    fc, se, conf = fitted.forecast(12, alpha=0.05)  # 95% conf
    
    # Make as pandas series
    fc_series = pd.Series(fc)
    lower_series = pd.Series(conf[:, 0])
    upper_series = pd.Series(conf[:, 1])
    
    for i in range(len(values)):
      fc[i]=values[i]
      #lower_series[i]=0
      #upper_series[i]=0
     
    cumulative_values.append(fc[0])
    lower_series[0]=fc[0]
    upper_series[0]=fc[0]
    for i in range(1,len(fc)):
        cumulative_values.append(fc[i] + cumulative_values[i-1])
        lower_series[i] += cumulative_values[i] - fc[i]
        upper_series[i] += cumulative_values[i] - fc[i]
        
    for i in range(len(values)):
      lower_series[i]=cumulative_values[i]
      upper_series[i]=cumulative_values[i]

    plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
    plt.hlines(target,0,len(cumulative_values)-1, 'r', 'dotted')
    plt.ylabel('Cumulative Measure Value')
    plt.plot(cumulative_values, '-bo')
    x_axis.extend(['February 2020','March 2020','April 2020','May 2020','June 2020'])
    plt.xticks(np.arange(len(x_axis)), x_axis, rotation='vertical')
    plt.show()
    
#generate_specific_metric_visualization('Number of Children at Lower Risk of Child Abuse','Expand Intervention Services for Family Violence')

In [0]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import TwoByTwoLayout
from IPython.display import clear_output

strategy_dropdown = widgets.Select(
    options=list(set(data['Strategy_Name'])),
    # rows=10,
    description='Strategy:',
    disabled=False
)
metric_dropdown = widgets.Select(
    options=list(set(data['Outcome_Indicator_Name'])),
    # rows=10,
    description='Metric:',
    disabled=False
)
program_dropdown = widgets.Select(
    options=list(set(data['Program_Name'])),
    # rows=10,
    description='Program:',
    disabled=False
)

**External - Public Facing Functionality**

In [0]:
btn = widgets.Button(description='Generate')
def btn_eventhandler(obj):
    #clear_output()
    generate_annual_metric_visualization(strategy_dropdown.value)
btn.on_click(btn_eventhandler)

TwoByTwoLayout(top_left=strategy_dropdown,
               top_right=btn)

In [0]:
btn = widgets.Button(description='Generate')
def btn_eventhandler(obj):
    generate_specific_metric_visualization(metric_dropdown.value,strategy_dropdown.value)
btn.on_click(btn_eventhandler)

display(btn)
TwoByTwoLayout(top_left=strategy_dropdown,
               top_right=metric_dropdown)

**Internal Facing Functionality**

In [0]:
btn = widgets.Button(description='Generate')
def btn_eventhandler(obj):
    #clear_output()
    generate_annual_metric_visualization(strategy_dropdown.value,program_dropdown.value)
btn.on_click(btn_eventhandler)

display(btn)
TwoByTwoLayout(top_left=strategy_dropdown,
               top_right=program_dropdown)

In [0]:
btn = widgets.Button(description='Generate')
def btn_eventhandler(obj):
    generate_specific_metric_visualization(metric_dropdown.value,strategy_dropdown.value,program_dropdown.value)
btn.on_click(btn_eventhandler)

TwoByTwoLayout(top_left=display(btn),
               bottom_left=strategy_dropdown,
               bottom_right=metric_dropdown,
               top_right=program_dropdown)